<a href="https://colab.research.google.com/github/kayleepierce/Clemson-University-Assignments/blob/main/HomeWork2_Part_B_Introduction_to_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/images/clemson_paw.png"> </div><br>


## Week 2: Introduction to Pandas

**Clemson University**<br>
**Spring 2024**<br>
**Instructor(s):** Nina Hubig <br>
**TA(s):** Luyi Li


---

In [1]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)

# Table of Contents
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6,000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, encoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

### Basic EDA workflow

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning.

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

##upload files to Google Colab

In [3]:
from google.colab import files
uploaded = files.upload()  #remember to click choose file, and click the goodreads.csv in the upload window

Saving goodreads.csv to goodreads (1).csv


### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [4]:
#Read the data into a dataframe
df = pd.read_csv("/content/goodreads.csv", encoding='utf-8')

#Examine the first few rows of the dataframe
df.head()

,4.4,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.Suzanne_Collins,2008,/genres/young-adult|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/book-club|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
0,4.41,16648.0,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478.0,Harry Potter and the Order of the Phoenix (Har...
1,3.56,85746.0,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564.0,"Twilight (Twilight, #1)"
2,4.23,47906.0,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123.0,To Kill a Mockingbird
3,4.23,34772.0,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992.0,Pride and Prejudice
4,4.25,12363.0,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470.0,Gone with the Wind


Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise 1</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [5]:
df=pd.read_csv("goodreads.csv", header=None,
               names=["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name'],
)

df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
0,4.40,136455.0,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008.0,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974.0,"The Hunger Games (The Hunger Games, #1)"
1,4.41,16648.0,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478.0,Harry Potter and the Order of the Phoenix (Har...
2,3.56,85746.0,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564.0,"Twilight (Twilight, #1)"
3,4.23,47906.0,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123.0,To Kill a Mockingbird
4,4.23,34772.0,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992.0,Pride and Prejudice


### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content.

<div class="exercise"><b>Exercise 2</b></div>
Lets check the types of the columns. What do you find?

In [6]:
df.dtypes

,0
rating,float64
review_count,float64
isbn,object
booktype,object
author_url,object
year,float64
genre_urls,object
dir,object
rating_count,float64
name,object


* textyour answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe.

In [7]:
print(df.shape)
df.columns

(6000, 10)


Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name'], dtype='object')

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [8]:
#Get a sense of how many missing values there are in the dataframe.
print(np.sum([df.rating.isnull()]))
print(np.sum([df.review_count.isnull()]))
print(np.sum([df.isbn.isnull()]))
print(np.sum([df.booktype.isnull()]))
print(np.sum([df.author_url.isnull()]))
print(np.sum([df.year.isnull()]))
print(np.sum([df.genre_urls.isnull()]))
print(np.sum([df.dir.isnull()]))
print(np.sum([df.rating_count.isnull()]))
print(np.sum([df.name.isnull()]))

2
2
477
2
2
7
62
0
2
2


In [9]:
#Try to locate where the missing values occur
df[df.rating.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
3643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dir37/9658936-harry-potter.html,NaN,NaN
5282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dir53/113138.The_Winner.html,NaN,NaN


How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [10]:
df[df.year.isnull()]

df.year.isnull()
df.shape

(6000, 10)

### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values?

In [11]:
#Treat the missing or invalid values in your dataframe
#######

df = df[df.year.notnull()]

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [12]:
df.dtypes

,0
rating,float64
review_count,float64
isbn,object
booktype,object
author_url,object
year,float64
genre_urls,object
dir,object
rating_count,float64
name,object


In [13]:
print(np.sum(df.year.isnull()))
df.shape # We removed seven rows

0


(5993, 10)

<div class="exercise"><b>Exercise 3</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [14]:
int_columns = ['rating', 'review_count', 'year', 'rating_count']
for col in int_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

    df[col] = df[col].fillna(0).astype(int)


<ipython-input-14-77199bb12d1a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')
<ipython-input-14-77199bb12d1a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(0).astype(int)
<ipython-input-14-77199bb12d1a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [15]:
df.dtypes

,0
rating,int64
review_count,int64
isbn,object
booktype,object
author_url,object
year,int64
genre_urls,object
dir,object
rating_count,int64
name,object


Sweet!

Some of the other colums that should be strings have NaN.

In [16]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [17]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/153394.Suzanne_Collins'

In [18]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

'Suzanne_Collins'

<div class="exercise"><b>Exercise 4</b></div>

Lets wrap the above code into a function which we will then use

In [19]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
def get_author(url):
    #Check if the url is a string
    if isinstance(url, str):
        name = url.split('/')[-1].split('.')[1:][0]
        return name
    # Return None if url is not a string
    else:
        return None

In [20]:
#Apply the get_author function to the 'author_url' column using '.map'
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

<ipython-input-20-9e2b8a22c7dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['author'] = df.author_url.map(get_author)


,author
0,Suzanne_Collins
1,J_K_Rowling
2,Stephenie_Meyer
3,Harper_Lee
4,Jane_Austen



Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [21]:

df.genre_urls.head()

,genre_urls
0,/genres/young-adult|/genres/science-fiction|/g...
1,/genres/fantasy|/genres/young-adult|/genres/fi...
2,/genres/young-adult|/genres/fantasy|/genres/ro...
3,/genres/classics|/genres/fiction|/genres/histo...
4,/genres/classics|/genres/fiction|/genres/roman...


In [22]:
#Examine some examples of genre_urls

#Test out some string operations to isolate the genre name
test_genre_string=df.genre_urls[0]
genres=test_genre_string.strip().split('|')
for e in genres:
    print(e.split('/')[-1])
    "|".join(genres)

young-adult
science-fiction
dystopia
fantasy
science-fiction
romance
adventure
book-club
young-adult
teen
apocalyptic
post-apocalyptic
action


<div class="exercise"><b>Exercise 5</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [23]:
def split_and_join_genres(url):
    genres=url.strip().split('|')
    genres=[e.split('/')[-1] for e in genres]
    return "|".join(genres)

Test your function

In [24]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

'young-adult|science-fiction'

In [25]:
split_and_join_genres("")

''

This question is primarily aimed at graduate students, serving as a critical part of their assignment. However, undergraduate students are encouraged to take this on as an optional bonus challenge.
<div class="exercise"><b>Exercise 6</b></div>
Use map again to create a new "genres" column

In [26]:
df['genres']=df.genre_url.map(split_and_join_genres)
df.head()

AttributeError: 'DataFrame' object has no attribute 'genre_url'

Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [27]:
df[df.author == "Marguerite_Yourcenar"]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author
1014,4,483,0374529264,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1951,/genres/historical-fiction|/genres/fiction|/ge...,dir11/12172.Memoirs_of_Hadrian.html,6258,Memoirs of Hadrian,Marguerite_Yourcenar
5620,4,74,2070367983,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1968,/genres/fiction|/genres/historical-fiction|/ge...,dir57/953435.L_uvre_au_noir.html,1601,L'Åuvre au noir,Marguerite_Yourcenar


Let us delete the `genre_urls` column.

In [28]:
del df['genre_urls']

And then save the dataframe out!

In [29]:
df.to_csv("/content/cleaned-goodreads.csv", index=False, header=True)

---

## Part 3: Grouping

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [30]:
df[df.year < 0].name
#These are books written before the Common Era (BCE, equivalent to BC).

,name
47,The Odyssey
246,The Iliad/The Odyssey
455,The Republic
596,The Aeneid
629,Oedipus Rex
674,The Art of War
746,The Bhagavad Gita
777,Antigone
1233,The Oedipus Cycle
1397,Aesop's Fables


We can determine the "best book" by year! For this we use Panda's `groupby()`. `Groupby()` allows grouping a dataframe by any (usually categorical) variable. Would it make sense to ever groupby integer variables? Floating point variables?

In [31]:
dfgb_author = df.groupby('author')
type(dfgb_author)

pandas.core.groupby.generic.DataFrameGroupBy

Perhaps we want the number of books each author wrote

In [32]:
dfgb_author.count()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name
author,,,,,,,,,
A_A_Milne,6,6,6,6,6,6,6,6,6
A_G_Howard,1,1,1,1,1,1,1,1,1
A_J_Cronin,1,1,1,1,1,1,1,1,1
A_J_Jacobs,1,1,1,1,1,1,1,1,1
A_J_Salt,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
_,42,42,42,42,42,42,42,42,42
_gota_Krist_f,1,1,1,1,1,1,1,1,1
_mile_Zola,4,4,4,4,4,4,4,4,4


Lots of useless info there. One column should suffice

### Exercise:

- Group the dataframe by `author`. Include the following columns: `rating`, `name`, `author`. For the aggregation of the `name` column which includes the names of the books create a list with the strings containing the name of each book. Make sure that the way you aggregate the rest of the columns make sense!

- Create a new column with number of books for each author and find the most prolific author!

In [33]:
###### Before we start : what do we do about these titles where 'name' is unreadable? Try different encodings?
auth_name = 'A_id_al_Qarni'
df[df.author == auth_name].head()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author
2213,4,1169,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2003,dir23/2750180.html,15781,ÙØ§ ØªØ­Ø²Ù,A_id_al_Qarni
5998,3,281,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2006,dir60/2750008.html,3083,Ø£Ø³Ø¹Ø¯ Ø§Ù Ø±Ø£Ø© ÙÙ Ø§ÙØ¹Ø§ÙÙ,A_id_al_Qarni


In [34]:
df[df.author == auth_name].iat[0,8].encode('UTF-16')

b'\xff\xfe\xd9\x00\x84\x00\xd8\x00\xa7\x00 \x00\xd8\x00\xaa\x00\xd8\x00\xad\x00\xd8\x00\xb2\x00\xd9\x00\x86\x00'

In [35]:
# let's examine the columns we have
df.columns

Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'dir', 'rating_count', 'name', 'author'], dtype='object')

Create the GroupBy table

In [36]:
authors = df.copy()
authors = authors[['rating','name','author']].groupby('author').agg({'rating' : np.mean,
                                                                    'name' : '|'.join})

<ipython-input-36-6b8bc2fc374f>:2: FutureWarning: The provided callable <function mean at 0x7eeb97d13d00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  authors = authors[['rating','name','author']].groupby('author').agg({'rating' : np.mean,


In [37]:
authors = authors.reset_index()
authors.head()

,author,rating,name
0,A_A_Milne,4.0,Winnie-the-Pooh|The House at Pooh Corner|The H...
1,A_G_Howard,4.0,"Splintered (Splintered, #1)"
2,A_J_Cronin,4.0,The Keys of the Kingdom
3,A_J_Jacobs,3.0,The Year of Living Biblically
4,A_J_Salt,4.0,Nik Nassa & the Mark of Destiny


In [38]:
# split the column string and make a list of string book names
authors['name'] = authors.name.str.split('|')
authors.head()

,author,rating,name
0,A_A_Milne,4.0,"[Winnie-the-Pooh, The House at Pooh Corner, Th..."
1,A_G_Howard,4.0,"[Splintered (Splintered, #1)]"
2,A_J_Cronin,4.0,[The Keys of the Kingdom]
3,A_J_Jacobs,3.0,[The Year of Living Biblically]
4,A_J_Salt,4.0,[Nik Nassa & the Mark of Destiny]


In [39]:
# count the books - create new column
len(authors.name[0])

6

In [40]:
authors['num_books'] = authors['name'].str.len()
authors

,author,rating,name,num_books
0,A_A_Milne,4.000000,"[Winnie-the-Pooh, The House at Pooh Corner, Th...",6
1,A_G_Howard,4.000000,"[Splintered (Splintered, #1)]",1
2,A_J_Cronin,4.000000,[The Keys of the Kingdom],1
3,A_J_Jacobs,3.000000,[The Year of Living Biblically],1
4,A_J_Salt,4.000000,[Nik Nassa & the Mark of Destiny],1
...,...,...,...,...
2640,_,3.571429,"[Ø¹Ø²Ø§Ø²ÙÙ, Ø«ÙØ§Ø«ÙØ© ØºØ±ÙØ§Ø·Ø©, ØªØ±...",42
2641,_gota_Krist_f,4.000000,"[The Notebook, The Proof, The Third Lie]",1
2642,_mile_Zola,3.750000,"[Germinal (Les Rougon-Macquart, #13), L'Assomm...",4
2643,_ric_Emmanuel_Schmitt,4.000000,[Oscar et la dame rose],1


In [41]:
# sort for more prolific
authors.sort_values(by='num_books', ascending=False).iloc[0]

,2349
author,Stephen_King
rating,3.375
name,"[The Stand, The Shining (The Shining #1), It, ..."
num_books,56


#### Winner is Stephen King with 56 books! OMG!!!

Perhaps you want more detailed info...

In [42]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

rating                                              rating_count                                                                             review_count                                                                      year                                                                   
                       count      mean      std  min   25%  50%  75%  max        count          mean           std      min       25%      50%       75%       max        count         mean          std     min      25%     50%      75%     max count         mean         std     min      25%     50%      75%     max
author                                                                                                                                                                                                                                                                                                                      
A_A_Milne                6.0  4.000000  0.00000  4.0  4.00  4.0  4.0  4.0          6.0  47842.333333  57135.314010    544.0  15559.50  30547.0  50401.25  157833.0          6.0   543.000000   682.473150     1.0   219.50   309.5   499.25  1886.0   6.0  1944.166667   29.294482  1926.0  1926.25  1927.5  1952.75  1997.0
A_G_Howard               1.0  4.000000      NaN  4.0  4.00  4.0  4.0  4.0          1.0  17073.000000           NaN  17073.0  17073.00  17073.0  17073.00   17073.0          1.0  3194.000000          NaN  3194.0  3194.00  3194.0  3194.00  3194.0   1.0  2013.000000         NaN  2013.0  2013.00  2013.0  2013.00  2013.0
A_J_Cronin               1.0  4.000000      NaN  4.0  4.00  4.0  4.0  4.0          1.0   1015.000000           NaN   1015.0   1015.00   1015.0   1015.00    1015.0          1.0   109.000000          NaN   109.0   109.00   109.0   109.00   109.0   1.0  1941.000000         NaN  1941.0  1941.00  1941.0  1941.00  1941.0
A_J_Jacobs               1.0  3.000000      NaN  3.0  3.00  3.0  3.0  3.0          1.0  39489.000000           NaN  39489.0  39489.00  39489.0  39489.00   39489.0          1.0  4371.000000          NaN  4371.0  4371.00  4371.0  4371.00  4371.0   1.0  2007.000000         NaN  2007.0  2007.00  2007.0  2007.00  2007.0
A_J_Salt                 1.0  4.000000      NaN  4.0  4.00  4.0  4.0  4.0          1.0     16.000000           NaN     16.0     16.00     16.0     16.00      16.0          1.0     6.000000          NaN     6.0     6.00     6.0     6.00     6.0   1.0  2014.000000         NaN  2014.0  2014.00  2014.0  2014.00  2014.0
...                      ...       ...      ...  ...   ...  ...  ...  ...          ...           ...           ...      ...       ...      ...       ...       ...          ...          ...          ...     ...      ...     ...      ...     ...   ...          ...         ...     ...      ...     ...      ...     ...
_                       42.0  3.571429  0.50087  3.0  3.00  4.0  4.0  4.0         42.0  11574.642857   9368.218450     10.0   4724.00   8709.5  15572.75   39302.0         42.0  1437.214286  1216.421371     1.0   534.75  1020.5  1745.75  4785.0  42.0  1998.904762   18.721071  1941.0  1994.75  2007.0  2010.75  2014.0
_gota_Krist_f            1.0  4.000000      NaN  4.0  4.00  4.0  4.0  4.0          1.0   2628.000000           NaN   2628.0   2628.00   2628.0   2628.00    2628.0          1.0   243.000000          NaN   243.0   243.00   243.0   243.00   243.0   1.0  1986.000000         NaN  1986.0  1986.00  1986.0  1986.00  1986.0
_mile_Zola               4.0  3.750000  0.50000  3.0  3.75  4.0  4.0  4.0          4.0   8358.250000   4553.226942   3594.0   5181.75   8095.0  11271.50   13649.0          4.0   283.750000   171.069917   125.0   189.50   243.5   337.75   523.0   4.0  1652.500000  456.345264   968.0  1649.75  1878.5  1881.25  1885.0
_ric_Emmanuel_Schmitt    1.0  4.000000      NaN  4.0  4.00  4.0  4.0  4.0          1.0   6439.000000           NaN   6439.0   6439.00   6439.0   6439.00    6439.0          1.0   424.000000          NaN   424.0   424.00

You can also access a `groupby` dictionary style.

In [43]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
ratingdict

{'A_A_Milne': (4.0, 0.0),
 'A_G_Howard': (4.0, nan),
 'A_J_Cronin': (4.0, nan),
 'A_J_Jacobs': (3.0, nan),
 'A_J_Salt': (4.0, nan),
 'A_Meredith_Walters': (4.0, 0.0),
 'A_N_Roquelaure': (3.0, 0.0),
 'A_S_Byatt': (3.0, nan),
 'A_S_King': (3.0, nan),
 'A_id_al_Qarni': (3.5, 0.7071067811865476),
 'Abbi_Glines': (3.857142857142857, 0.3631365196012815),
 'Abdul_Rahman_Munif': (4.0, nan),
 'Abigail_Gibbs': (3.0, nan),
 'Abigail_Roux': (4.0, 0.0),
 'Abigail_Thomas': (3.0, nan),
 'Abolqasem_Ferdowsi': (4.0, nan),
 'Abraham_Verghese': (4.0, nan),
 'Abul_Hasan_Ali_Nadwi': (4.0, nan),
 'Adam_Hochschild': (4.0, nan),
 'Adam_Johnson': (4.0, nan),
 'Adam_Levin': (4.0, nan),
 'Adam_Rex': (4.0, nan),
 'Adam_Smith': (3.0, nan),
 'Addison_Moore': (3.0, nan),
 'Adeline_Yen_Mah': (4.0, nan),
 'Adolf_Hitler': (2.0, nan),
 'Adolfo_Bioy_Casares': (4.0, nan),
 'Aeschylus': (3.0, nan),
 'Aesop': (4.0, nan),
 'Agatha_Christie': (3.3636363636363638, 0.504524979109513),
 'Ahlam_Mosteghanemi': (3.0, nan),
 'Ahmad_

**This question is primarily aimed at graduate students, serving as a critical part of their assignment. However, undergraduate students are encouraged to take this on as an optional bonus challenge.**
<div class="exercise"><b>Exercise 7</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [44]:
#Using .groupby, we can divide the dataframe into subsets by the values of 'year'.
#We can then iterate over these subsets
# your code here
for year, subset in df.groupby('year'):
    #Find the best book of the year

    bestbook = subset[subset.rating == subset.rating.max()]
    if bestbook.shape[0] > 1:
        print(year, bestbook.name.values, bestbook.rating.values)
    else:
        print(year, bestbook.name.values[0], bestbook.rating.values[0])

-1500 The Epic of Gilgamesh 3
-800 The Iliad/The Odyssey 4
-560 Aesop's Fables 4
-512 The Art of War 3
-500 The Bhagavad Gita 4
-458 The Oresteia 3
-442 Antigone 3
-440 The Histories 3
-431 Medea 3
-429 Oedipus Rex 3
-411 ['The History of the Peloponnesian War' 'Lysistrata'] [3 3]
-400 ['The Trial and Death of Socrates' 'Complete Works'] [4 4]
-390 Apology 4
-380 The Republic 3
-370 The Symposium 3
-360 Five Dialogues 4
-350 The Nicomachean Ethics 3
-29 The Aeneid 3
4 The Seven Spiritual Laws of Success 4
8 Metamorphoses 4
13 The Day Jesus Rode Into Croydon 3
100 The New Oxford Annotated Bible, New Revised Standard Version 4
121 The Twelve Caesars 4
130 Holy Bible 4
158 The Golden Ass 3
174 Meditations 4
200 The Garden of Eden 3
201 Fe Fi FOE Comes 4
213 NaÃ¯ve. Super 3
397 Confessions 3
632 The Quran 4
800 Beowulf 3
900 The Arabian Nights 4
968 La BÃªte humaine (Les Rougon-Macquart, #17) 4
1002 The Pillow Book 4
1008 The Tale of Genji 3
1010 Shahnameh 4
1100 Rubaiyat of Omar Khayyam 4